In [1]:
pip install pinecone

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 10.5 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pinecone import Pinecone


In [ ]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
pc = Pinecone(api_key="pcsk_5RSTcB_8GfPp22T2dhbtGz59ihsbPZQnG6HHnia2QJosnBy22T8ZTu1wdcwhqdWqhRrUcm")


pc.create_index(
  name="teste",
  dimension=1536,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

{
    "name": "teste-pinecone",
    "metric": "cosine",
    "host": "teste-pinecone-kg31ntv.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:
index = pc.Index(
    name="teste" # <-- make sure to set these
)

In [ ]:
import cohere
co = cohere.Client(api_key="RAjqpE9uTVFfTTaF0RIGUM4PaKBeQ7insNIu9KLC") # Cliente do Cohere


# Pergunta do usuário
pergunta_usuario = "Você é uma IA que deve falar de aliemntação, como posso saber se estou me alimentando direito?"

# 1. EMBEDDING DA PERGUNTA (usando o input_type correto)
response_query = co.embed(
    texts=[pergunta_usuario], 
    model='embed-multilingual-v3.0',
    input_type='search_query' # <--- MUITO IMPORTANTE!
)

vetor_da_pergunta = response_query.embeddings[0]

# 2. BUSCA NO PINECONE (esta parte não muda)
resultados_pinecone = index.query(
    vector=vetor_da_pergunta,
    top_k=3, # Pega os 3 chunks mais relevantes
    include_metadata=True
)

# 3. MONTAR O CONTEXTO PARA O COMMAND-R
contexto = ""
documentos_recuperados = []
for match in resultados_pinecone['matches']:
    contexto += match['metadata']['texto'] + "\n---\n"
    documentos_recuperados.append({"id": match['id'], "text": match['metadata']['texto']})


# 4. GERAR A RESPOSTA FINAL COM O COHERE COMMAND-R

resposta_final = co.chat(
    model="command-r",
    message=pergunta_usuario,
    documents=documentos_recuperados, # Cohere otimiza para isso
    preamble="Você é um assistente especialista em legislação brasileira. Responda à pergunta do usuário baseando-se estritamente nos documentos fornecidos."
)


# Exibir a resposta
print("Resposta Gerada pelo Cohere Command-R:")
print(resposta_final.text)

Resposta Gerada pelo Cohere Command-R:
De acordo com a legislação brasileira, o Ministério da Saúde possui um guia alimentar para a população chamado "Guia Alimentar para a População Brasileira". Ele fornece recomendações sobre os grupos alimentares que devem ser consumidos diariamente para uma alimentação saudável e equilibrada. 

Aqui estão algumas das principais recomendações do guia:

1 - Compre alimentos de todos os grupos alimentares: Frutas, hortaliças, arroz e feijões, cereais, tubérculos, carnes, ovos, leite e derivados, e óleos e gorduras.

2 - Dê prioridade a alimentos in natura ou minimamente processados: Alimentos frescos, cozidos, assados ou fervidos, são geralmente mais saudáveis do que os processados.

3 - Consuma pelo menos 3 porções de frutas e hortaliças por dia: Uma porção pode ser um fruto médio, uma xícara de folhas verdes, meia xícara de legumes cozidos, entre outros.

4 - Opte por alimentos integrais: Arroz, pão, macarrão e farinhas integrais são fontes de carbo

In [ ]:
import os
import cohere
import pinecone
from pinecone import Pinecone
from typing import List, Dict


def buscar_e_responder(pergunta) -> Dict:

    # --- 1. PARÂMETROS E CONFIGURAÇÕES INTERNAS ---

    # AVISO: É uma má prática de segurança deixar chaves no código.
    # O ideal é usar variáveis de ambiente: os.getenv("SUA_CHAVE")
    COHERE_API_KEY = "RAjqpE9uTVFfTTaF0RIGUM4PaKBeQ7insNIu9KLC"
    PINECONE_API_KEY = "pcsk_5RSTcB_8GfPp22T2dhbtGz59ihsbPZQnG6HHnia2QJosnBy22T8ZTu1wdcwhqdWqhRrUcm"
    PINECONE_INDEX_NAME = "teste"

    # Parâmetros da busca e da geração
    TOP_K = 5
    TEMPERATURE = 0.3

    # Personalidade da IA
    PERSONA = """
**Você é "Nutri-Cortex", um Assistente de IA de suporte à decisão para nutricionistas.**

Sua função é processar, correlacionar e resumir informações da base de dados científicos e dos prontuários de pacientes. Você existe para otimizar o tempo do nutricionista, destacando informações relevantes.

**Regras Inquebráveis:**
1. **Posicionamento de Assistente:** Suas respostas são informações para a avaliação do profissional. A decisão final é 100% do nutricionista.
2. **Fidelidade ao Contexto:** Sua resposta DEVE ser gerada EXCLUSIVAMENTE a partir dos documentos fornecidos. É PROIBIDO usar conhecimento externo.
3. **Cite Suas Fontes:** Sempre que possível, referencie a origem da sua informação.
4. **Informação Faltante:** Se a informação não estiver nos documentos, afirme claramente: "A informação necessária para responder a esta pergunta não foi encontrada nos documentos fornecidos."
5. **Objetividade e Precisão:** Apresente os dados de forma neutra, técnica e profissional.
"""

    try:
        # --- 2. INICIALIZAÇÃO DOS CLIENTES (DENTRO DA FUNÇÃO) ---
        cliente_cohere = cohere.Client(api_key=COHERE_API_KEY)
        pc = Pinecone(api_key=PINECONE_API_KEY)
        indice_pinecone = pc.Index(name=PINECONE_INDEX_NAME)
        # print("Clientes internos inicializados.") # Descomente para depurar

        # --- 3. LÓGICA RAG ---

        # Embedding da pergunta
        response_query = cliente_cohere.embed(
            texts=[pergunta],
            model='embed-multilingual-v3.0',
            input_type='search_query'
        )
        vetor_da_pergunta = response_query.embeddings[0]

        # Busca no Pinecone
        resultados_pinecone = indice_pinecone.query(
            vector=vetor_da_pergunta,
            top_k=TOP_K,
            include_metadata=True
        )


        # Montar o contexto
        documentos_recuperados = []
        for match in resultados_pinecone['matches']:
            documentos_recuperados.append({
                "id": match['id'],
                "text": match['metadata'].get('text', ''),
                "source": match['metadata'].get('file_name', 'N/A'),
                "page_label": match['metadata'].get('page_label', '')
            })

        # Geração da resposta
        resposta_final = cliente_cohere.chat(
            model="command-r",
            message=pergunta,
            documents=documentos_recuperados,
            preamble=PERSONA,
            temperature=TEMPERATURE
        )

        # Retorna o resultado
        return {
            "resposta": resposta_final.text,
            "fontes": documentos_recuperados
        }
    except Exception as e:
        print(f"Ocorreu um erro dentro da função buscar_e_responder: {e}")
        return {
            "resposta": "Desculpe, ocorreu um erro ao processar sua solicitação.",
            "fontes": []
        }


# --- COMO USAR A FUNÇÃO ---

# 1. Defina a pergunta (a única variável fora da função)
minha_pergunta = "Qual o protocolo de reintrodução alimentar (refeeding) mais seguro para um paciente com anorexia nervosa e baixo peso severo, detalhando a oferta calórica inicial e a progressão para evitar a síndrome de realimentação?"

# 2. Chame a função passando apenas a pergunta
resultado_completo = buscar_e_responder(minha_pergunta)

# 3. Exiba o resultado
print("\n--- Resposta Gerada ---")
print(resultado_completo["resposta"])

print("\n--- Fontes Utilizadas ---")
if resultado_completo["fontes"]:
    for fonte in resultado_completo["fontes"]:
        print(f"- ID: {fonte['id']}, Fonte: {fonte.get('source', 'N/A')}, Pagina:{fonte.get('page_label', 'N/A')}")
else:
    print("Nenhuma fonte foi recuperada.")



--- Resposta Gerada ---
O protocolo de reintrodução alimentar, ou refeeding, para pacientes com anorexia nervosa e baixo peso severo consiste em uma abordagem gradual e monitorada. A oferta calórica inicial recomendada é de aproximadamente 1000 a 1500 kcal diárias, que devem ser divididas em 3 a 6 refeições, sendo 3 principais e 3 lanches. Essa fase inicial deve durar de 2 a 4 dias, e a progressão para a próxima etapa deve ser feita com cuidado e baseando-se na tolerância individual do paciente.

Durante esse período inicial, é importante monitorar qualquer sinal ou sintoma de distensão abdominal, náusea, vômito ou desconforto gastrointestinal, pois eles podem indicar que a progressão está ocorrendo muito rapidamente. Caso esses sintomas sejam observados, deve-se reduzir o ritmo de aumento da oferta calórica.

Após a fase inicial, a oferta calórica deve ser aumentada gradualmente, em cerca de 500 kcal por dia, até que se alcance um valor próximo das 3000 kcal diárias. Novamente, a tol

In [29]:

pergunta = "Qual o protocolo de reintrodução alimentar (refeeding) mais seguro para um paciente com anorexia nervosa e baixo peso severo, detalhando a oferta calórica inicial e a progressão para evitar a síndrome de realimentação?"

resultado_completo = buscar_e_responder(pergunta, "teste")

print("--- Resposta Gerada ---")
print(resultado_completo["resposta"])
print("\n--- Fontes Utilizadas ---")
for fonte in resultado_completo["fontes"]:
    print(f"- ID: {fonte['id']}, Fonte: {fonte.get('source', 'N/A')}")

--- Resposta Gerada ---
## Solicitação
O nutricionista solicita o protocolo de reintrodução alimentar seguro para paciente com anorexia nervosa e baixo peso severo, focusing na oferta calórica inicial e progressão do tratamento.

## Análise dos Dados do Paciente
O prontuário indica baixo peso severo e diagnóstico de anorexia nervosa. O paciente apresenta histórico de perda de peso drástica por meio de dieta restritiva e exercícios intensos. Exames físicos revelam: peso corporal muito baixo, massa muscular reduzida, mas exames laboratoriais dentro dos parâmetros normais. 

## Correlação com a Evidência Científica
A síndrome de realimentação é um risco sério em casos de baixo peso severo. O refeeding deve ser cuidadosamente planejado e monitorado. A literatura científica sugere que uma abordagem gradual e multidisciplinar é a mais segura, com monitoramento constante dos sinais vitais e resposta individual do paciente. 

A oferta calórica inicial deve ser de baixa intensidade, entre 500-1